In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization,Flatten,\
Add,Input,Dense, Dropout, Activation, InputLayer
from tensorflow.keras.optimizers import Adam



from augmentation_code import data_augmentation
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os 
from sklearn.model_selection import KFold

In [2]:
def set_globals():
    global open3D_directory
    global dataset_name_num
    global MIF_nodes 
    global xyz_offset
    open3D_directory = "C:\open3dtools"
    dataset_name_num = "06.DHFR"
    MIF_nodes = [23,27,21]
    xyz_offset = [-11,-10,-9]
set_globals()

In [3]:
def import_data():
    path = os.path.join(r"C:\Users\Linden\GitHub\3DQSAR\data\LabelledData",dataset_name_num)
    labels = pd.read_csv(os.path.join(path,"pIC50.CSV"))
    Y = labels["Y"].to_numpy()

    data=[]

    for entry in os.scandir(path):
        if (entry.path.endswith(".npy")):
            file = np.load(entry.path)
            data.append(file)
    field_1 = data[0::2]
    field_2 = data[1::2]
    X_1 = np.stack(field_1,0)
    X_2 = np.stack(field_2,0)    
    
    return X_1, X_2, Y

In [36]:
X_1, X_2, Y = import_data()

In [ ]:
X_1

In [42]:
X_1[285].shape

(25, 24, 26)

In [110]:
####Augment and extend data
X_1, X_2, Y = data_augmentation(X_1,X_2,Y,augment_repetitions=5)

In [ ]:
X_1[]

In [25]:
###Pickle data

In [26]:
###Normalise image data 
###Normalise by amax value?

In [27]:
X_1.shape

(306, 25, 24, 26)

In [28]:
###Reshape
X_1 = np.reshape(X_1,(X_1.shape[0],X_1.shape[1],X_1.shape[2],X_1.shape[3],1))
X_2 = np.reshape(X_2,(X_2.shape[0],X_2.shape[1],X_2.shape[2],X_2.shape[3],1))

In [29]:
###Shuffle
permutation = np.random.RandomState(seed=42)\
                .permutation(X_1.shape[0])

X_1 = X_1[permutation]
X_2 = X_2[permutation]
Y = Y[permutation]

In [30]:
####Train test split
train_upper = int(np.around(X_1.shape[0]*0.9,0))

X_1_train = X_1[:train_upper]
X_2_train = X_2[:train_upper]
X_1_test = X_1[train_upper:]
X_2_test = X_2[train_upper:]

Y_train = Y[:train_upper]
Y_test = Y[train_upper:]

In [31]:
###Train validation split
train_upper = int(np.around(X_1_train.shape[0]*0.8,0))
X_1_val = X_1_train[train_upper:]
X_2_val = X_2_train[train_upper:]
X_1_train = X_1_train[:train_upper]
X_2_train = X_2_train[:train_upper]

Y_val = Y_train[train_upper:]
Y_train = Y_train[:train_upper]


In [32]:
def build_CNN():
    model = Sequential(name='CNN')
    
    # elu=Exponential Linear Unit, similar to leaky Relu
    #perhaps normalise in layer 1 
    
    # Convolution Layers
    inputs = Input(shape=(X_1_train.shape[1], X_1_train.shape[2],X_1_train.shape[3],X_1_train.shape[4]))
    model = MaxPooling3D(pool_size=(2,2,2))(inputs)
    model = Conv3D(32, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Conv3D(64, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Conv3D(128, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    
    model = Flatten()(model)
    model = Dense(15,activation='relu')(model)
    output = model
    
    model = Model(inputs=inputs, outputs=output)
    return model

In [33]:
def two_headed_network():
    X_1_CNN = build_CNN()
    X_2_CNN = build_CNN()
    network = Add()([X_1_CNN.output,X_2_CNN.output])
    network = Flatten()(network)
    network = Dense(10,activation='relu')(network)
    network = Dense(1,activation = 'linear')(network)
    model = Model([X_1_CNN.input, X_2_CNN.input], network)
    
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    return model  
CNN = two_headed_network()

In [34]:
CNN.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 25, 24, 26,  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 25, 24, 26,  0                                            
__________________________________________________________________________________________________
max_pooling3d_13 (MaxPooling3D) (None, 12, 12, 13, 1 0           input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling3d_16 (MaxPooling3D) (None, 12, 12, 13, 1 0           input_7[0][0]                    
____________________________________________________________________________________________

In [109]:
#####Cross validation
perm = np.random.RandomState(seed=42)\
                .permutation(X_1_train.shape[0])
num_folds=5
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
monitor = EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=15,
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True)
for perms_train,perms_test in kfold.split(X=perm,y=perm):
    history = CNN.fit([X_1_train[perms_train], X_2_train[perms_train]],
                       Y_train[perms_train],
                       batch_size=16,
                       epochs=200, 
                       validation_data=([X_1_train[perms_test], X_2_train[perms_test]],
                                         Y_train[perms_test]),
                                         callbacks=[monitor])

Epoch 1/200


KeyboardInterrupt: 

In [108]:
perms_test

array([  1,   2,   3,   8,   9,  12,  17,  19,  30,  32,  39,  42,  46,
        50,  54,  55,  56,  62,  64,  68,  76,  80,  83,  91, 101, 124,
       126, 131, 135, 143, 147, 151, 154, 167, 177, 179, 182, 195, 196,
       197, 200, 201, 203, 219])

In [105]:
##Train 
##monitor early stopping using val_acc
#Set min_delta to 1e-3
monitor = EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=15,
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True)

CNN.fit([X_1_train, X_2_train],
        Y_train,
        batch_size=16,
        epochs=200, 
        validation_data=([X_1_val, X_2_val],
                          Y_val),
                          callbacks=[monitor])

Epoch 1/200


KeyboardInterrupt: 

In [ ]:
##Test model w/ best weights on validation data
model_loss = CNN.evaluate([X_1_test,X_2_test], Y_test)
print(f"Model loss:{model_loss}")
